In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [17]:
import nltk
# Launch the installer to download "gutenberg" and "stop words" corpora.
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> gutenberg
      Package gutenberg is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Package stopwords is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [0]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [0]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [20]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

print(len(alice_sents))
print(len(persuasion_sents))

1669
3649


In [21]:
#due to computational limitations, reduce length of alice_sents to 1000
alice_sents = alice_sents[0:1000]
persuasion_sents = persuasion_sents[0:1000]

print(len(alice_sents))
print(len(persuasion_sents))

1000
1000


In [22]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [0]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [24]:
# Create our data frame with features. This can take a while to run.
import time
start_time = time.time()
word_counts = bow_features(sentences, common_words)
print("-- Execution time: %s seconds ---" % (time.time() - start_time))
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
-- Execution time: 4782.072224378586 seconds ---


,bird,calmly,season,noise,dry,ambition,recover,evening,between,uppercross,...,sorrowful,drown,sky,plan,give,deserted,est,happiness,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [25]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9875

Test set score: 0.84875


In [26]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(1200, 3062) (1200,)
Training set score: 0.9716666666666667

Test set score: 0.87875


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8808333333333334

Test set score: 0.80625


In [28]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [0]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [0]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [31]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
done


In [32]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6633102580239144


col_0,Austen,Carroll
row_0,,
Austen,772,228
Carroll,307,282


## Challenge 0: 

Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

In [0]:
## Feature Eng. 

#word count per sentence
sentences = word_counts.text_sentence
word_counts['sentence_len'] = [len(i) for i in sentences]

#noun count per sentence
nouns = []
for sent in sentences: 
  noun = 0 
  for token in sent: 
    noun = noun + token.pos_.count('NOUN')
  word_counts['nouns'] = noun 

#verb count per sentence
verbs = []
for sent in sentences: 
  verb = 0 
  for token in sent: 
    verb = verb + token.pos_.count('VERB')
  word_counts['verbs'] = verb 
   
#adjective count per sentence
adjectives = []
for sent in sentences: 
  adjective = 0 
  for token in sent: 
    adjective = adjective + token.pos_.count('ADJ')
  word_counts['adjectives'] = adjective
   
#proper noun count per sentence
propers = []
for sent in sentences: 
  proper = 0 
  for token in sent: 
    proper = proper + token.pos_.count('PROPN')
  word_counts['propers'] = proper
  
#punctuation count per sentence
punctuations = []
for sent in sentences: 
  punctuation = 0 
  for token in sent: 
    punctuation = punctuation + token.pos_.count('PUNCT')
  word_counts['punctuations'] = punctuation  

In [43]:
##Check updated DF 
word_counts.head()

,bird,calmly,season,noise,dry,ambition,recover,evening,between,uppercross,...,est,happiness,text_sentence,text_source,sentence_len,nouns,verbs,punctuations,adjectives,propers
0,0,0,0,0,0,0,0,0,0,0,...,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67,3,5,3,2,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,63,3,5,3,2,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33,3,5,3,2,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,"(Oh, dear, !)",Carroll,3,3,5,3,2,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,"(I, shall, be, late, !, ')",Carroll,6,3,5,3,2,0


In [0]:
#Split data for training purposes
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [45]:
from sklearn.model_selection import cross_val_score
#fit logistic regression with updated DF 
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(lr, X_test, y_test, cv=5))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(1200, 3068) (1200,)
Training set score: 0.9808333333333333

Test set score: 0.87625


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)



Cross Val score: [0.85714286 0.90625    0.84375    0.90625    0.8490566 ]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Challenge 1:

Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [71]:
# Bring in/clean the shakespeare data.
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
hamlet = re.sub(r'VOLUME \w+', '', hamlet)
hamlet = re.sub(r'CHAPTER \w+', '', hamlet)
hamlet = text_cleaner(hamlet)
print(hamlet[:100])

Actus Primus. Scoena Prima. Enter Barnardo and Francisco two Centinels. Barnardo. Who's there? Fran.


In [72]:
print(len(hamlet))

158933


In [0]:
# Parse our cleaned data.
hamlet_doc = nlp(hamlet)

In [75]:
# Group into sentences.
hamlet_sents = [[sent, "Shakespeare"] for sent in hamlet_doc.sents]

print(len(hamlet_sents))

3039


In [0]:
# Due to computational limitations, reduce length of hamlet_sents to 1000
hamlet_sents = hamlet_sents[0:1000]

In [77]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + hamlet_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [0]:
# Set up the bags of words.
hamletwords = bag_of_words(hamlet_doc)
common_words = set(alicewords+hamletwords)

In [82]:
# Build a new Bag of Words data frame for Hamlet word counts.
# We'll use the same common words from Alice and Persuasion.
hamlet_bow = bow_features(sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [83]:
hamlet_bow.head()

,bird,calmly,season,noise,dry,dreadfull,ambition,recover,evening,between,...,sorrowful,drown,sky,plan,give,elsonower,aloofe,est,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [0]:
## Same feature eng. as previously done 

#word count per sentence
sentences = hamlet_bow.text_sentence
hamlet_bow['sentence_len'] = [len(i) for i in sentences]

#noun count per sentence
nouns = []
for sent in sentences: 
  noun = 0 
  for token in sent: 
    noun = noun + token.pos_.count('NOUN')
  hamlet_bow['nouns'] = noun 

#verb count per sentence
verbs = []
for sent in sentences: 
  verb = 0 
  for token in sent: 
    verb = verb + token.pos_.count('VERB')
  hamlet_bow['verbs'] = verb 
  
#adjective count per sentence
adjectives = []
for sent in sentences: 
  adjective = 0 
  for token in sent: 
    adjective = adjective + token.pos_.count('ADJ')
  hamlet_bow['adjectives'] = adjective
   
#proper noun count per sentence
propers = []
for sent in sentences: 
  proper = 0 
  for token in sent: 
    proper = proper + token.pos_.count('PROPN')
  hamlet_bow['propers'] = proper
  
#punctuation count per sentence
punctuations = []
for sent in sentences: 
  punctuation = 0 
  for token in sent: 
    punctuation = punctuation + token.pos_.count('PUNCT')
  hamlet_bow['punctuations'] = punctuation

In [0]:
Y = hamlet_bow['text_source']
X = np.array(hamlet_bow.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [86]:
#fit logistic regression with hamlet data 
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(lr, X_test, y_test, cv=5))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(1200, 3360) (1200,)
Training set score: 0.9683333333333334

Test set score: 0.8975


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur


Cross Val score: [0.90062112 0.85       0.875      0.89375    0.89937107]


In [0]:
#t-SNE data visualization with words etc. 